# Evaluating numerical expressions

In this project, I will use stacks to implement an algorithm that can evaluate numerical expressions.

In [2]:
# importing a linked list class
from linked_list import LinkedList

In [5]:
# Creating a stack class
class Stack(LinkedList):
    def push(self, data):
        self.append(data)
    def peek(self):
        return self.tail.data
    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Implementing functions to evaluate expressions in postfix notation

When we write an expression, we use infix notation, meaning that we put the operators between the two operands. For example 1 + 2 is in infix notation because the + operator is between the operands 1 and 2.

For a computer, it's much easier to evaluate an expression written in postfix notation. In postfix notation, the operands appear before the operator. The infix expression 1 + 2 becomes 1 2 + in postfix notation.

To simplify the function, we will assume that, in the expression string we want to evaluate, there are spaces between all elements of the expression.

Based on this assumption, we can transform the postfix expression string into a list of elements using the str.split() method. In the context of evaluating expressions, we call these elements tokens, and the term for transforming the expression into a list of tokens is tokenize. 

In [6]:
# Function to tokenize a postfix expression
def tokenize(string):
    return string.split()

# Testing the function
test = '1 5 * 2 / 4 5 +'
tokenize(test)

['1', '5', '*', '2', '/', '4', '5', '+']

Evaluating an expression in postfix notation using a stack involves the following:
- If we find a number, then we push that number to the top of the stack.
- If we find an operator, we pop the top two elements of the stack, perform the operation, and then push back the result. The element at the top of the stack is actually the second operand in the expression, while the element that is second to top is the first operand.

In [16]:
# function for processing - token
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top
    stack.push(result)

In [17]:
# function for processing + token
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top
    stack.push(result)

In [18]:
# function for processing * token
def process_mult(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top
    stack.push(result)

In [19]:
# function for processing / token
def process_div(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top
    stack.push(result)

In [20]:
# function for processing ** token
def process_power(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top
    stack.push(result)

## Algorithm for evaluating postfix expression

In [24]:
def evaluate_postfix(expression):
    tokenized = tokenize(expression)
    stack = Stack()
    for t in tokenized:
        if t == "+":
            process_plus(stack)
        elif t == "-":
            process_minus(stack)
        elif t == "*":
            process_mult(stack)
        elif t == "/":
            process_div(stack)
        elif t == "**":
            process_power(stack)
        else:
            # The token is not an operator so it must be a number
            stack.push(float(t))
    return stack.pop()
            

In [25]:
# testing the algorithm
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

In [26]:
answers = []
for e in expressions:
    answer = evaluate_postfix(e)
    answers.append(answer)
answers

[-2.0, 8.0, 0.0, 2.0, 11.25, 45.0, 42.0, 4.0, 2.0]

### Converting infix expressions to postfix

I'll implement the Shunting-yard algorithm. The data structure to implement this algorithm is (again) a stack.

In the Shunting-yard algorithm, we'll need to compare the precedence of the operators in infix notation. We will use numbers to represent the operator precedence. The higher the precedence, the higher the number. 

- 1 for + & -
- 2 for * & /
- 3 for **

In [28]:
# creating a precedence dictionary
precedence = {
    '+': 1,
    '-': 1,
    '*': 2,
    '/': 2,
    '**': 3
}

# testing the dictionary
print(precedence["/"] < precedence["-"])
print(precedence["/"] < precedence["**"])
print(precedence["+"] < precedence["*"])

False
True
True


Now I'll create a function for converting infix to postfix notation. The function will start by tokenizing the postfix expression, and then it processes the tokens one by one using a stack. It builds the postfix expression by keeping track of a list named postfix, which will contain the list of tokens in postfix order.

Here's how processing should go for each token:

- Opening parenthesis, (:
    1. Push the token into the stack for later use when we find a closing parenthesis.
    
    
- Closing parenthesis ):
    1. While the top of the stack isn't an opening parenthesis, (, pop the top element, and append it to the postfix token list.
    2. Pop the opening parentheses out of the stack at the end.
    
    
- Operator, +, -, *, /, or **:
    1. While the top of the stack is also an operator with a precedence greater than or equal to this operator, pop the top element, and append it to the postfix token list.
    2. Push the current operator to the top of the stack
    
    
- Operand (any number):
    1. Append the number to the postfix token list.
    
After having processed all tokens, the stack may not be empty. In this case we pop all remaining operators into the postfix list.

In [42]:
# Function for handling open parenthesis
def process_opening_parenthesis(stack):
    stack.push("(")

# Function for handling closing parenthesis
"""Logic: While the top of the stack isn't an opening parenthesis, (, 
pop the top element, and append it to the postfix token list. 
Pop the opening parentheses out of the stack at the end."""
def process_closing_parenthesis(stack, postfix):
    while stack.peek() != "(":
        postfix.append(stack.pop())
    stack.pop()

# Function for handling operators
"""Logic: While the top of the stack is also an operator 
with a precedence greater than or equal to this operator, 
pop the top element and append it to the postfix token list.
Push the current operator to the top of the stack.
"""
def process_operator(stack, postfix, operator):
    while len(stack) > 0 and stack.peek() in precedence and  precedence[stack.peek()] >= precedence[operator]:
    # Pop the top of the stack into the postfix list
        postfix.append(stack.pop())
    stack.push(operator)

# Function for handling numbers
"""Logic: Push the token into the postfix token list"""
def process_number(postfix, number):
    postfix.append(number)


### Putting together an infix_to_postfix() function

This function will work as follows:

1. We start by splitting the expression into tokens using the tokenize() function.
2. We initialize an empty stack.
3. We initialize an empty postfix token list.
4. Iterate over all tokens, and for each, do the following:
    - If the token is "(", we call the process_opening_parenthesis() function.
    - If the token is ")", we call the process_closing_parenthesis() function.
    - If the token is an operator, we call the process_operator() function.
    - Otherwise, the token is a number, and we call the process_number() function.
5. After processing all tokens, we use a while loop to pop the remaining stack element into the postfix token list.
6. Use the str.join() method to convert the postfix token list into a string.

In [43]:
def infix_to_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    postfix = []
    for token in tokens:
        if token == "(":
            process_opening_parenthesis(stack)
        elif token == ")":
            process_closing_parenthesis(stack, postfix)
        elif token in precedence:
            process_operator(stack, postfix, token)
        else:
            process_number(postfix, token)
    while len(stack)>0:
        postfix.append(stack.pop())
    return " ".join(postfix)

### Combining the 2 functions to evaluate numerical expressions

In [44]:
def evaluate(expression):
    postfix_expression = infix_to_postfix(expression)
    return evaluate_postfix(postfix_expression)

In [46]:
# Testing the function
expressions = [
    "1 + 1",
    "1 * ( 2 - ( 1 + 1 ) )",
    "4 * ( 1 + 2 * ( 9 / 3 ) - 5 )",
    "10 + 3 * 5 / ( 16 - 4 * 1 )",
    "2 * 2 * 2 * 2 * 2 * 2 * 2 * 2",
    "2 ** 2 ** 2 ** 2 ** 2",
    "( 1 - 2 ) / ( 3 - 5 )",
    "9 / 8 * 8",
    "64 / ( 8 * 8 )",
]

results = []
for e in expressions:
    result = evaluate(e)
    results.append(result)
    
print(results)

[2.0, 0.0, 8.0, 11.25, 256.0, 65536.0, 0.5, 9.0, 1.0]


The evaluate() function that we implemented exists in Python as the eval() built-in function. The Python implementation can actually evaluate any string of Python code, not only expressions.

Possible next steps:

- Improving the tokenize() method to make it more robust as one of the limitations of our implementation is that it requires spaces to separate every part of the expression. For example, we can evaluate the expression 2 * ( 5 - 3 ), but we can't evaluate 2 * (5 - 3) or 2*(5 - 3). 

- Implement other operators — for example, the integer division //.